## Lecture 06 Keywords Extraction & Search Engine

Outline:
* Finding similar words using Word2Vec;
* TF-IDF Keyword Extraction;
* Word cloud;
* Build a search engine
     + input: words;
     + output: matched documents;
     + two ranking methods: 1) ranked by TF-IDF; 2) page rank

In [18]:
csv_path = '/Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/data/sqlResult_1558435.csv'

In [19]:
import pandas as pd

In [20]:
content = pd.read_csv(csv_path, encoding='gb18030')

In [21]:
content = content.fillna('') # remove NAs

In [22]:
news_content = content['content'].tolist()

In [23]:
import jieba

def cut(string): return ' '.join(jieba.cut(string))

In [24]:
cut('这是一个测试')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/1y/1btp7xpj7b1f82lnwvn2916h0000gn/T/jieba.cache
Loading model cost 0.688 seconds.
Prefix dict has been built succesfully.


'这是 一个 测试'

In [25]:
import re

def find_tokens(string):
    "a function to find all the valide tokens"
    return re.findall(r'[\d|\w]+', string)

In [26]:
find_tokens('这是一个测试\n\n\n')

['这是一个测试']

In [27]:
news_content = [find_tokens(n) for n in news_content]

In [28]:
news_content = [' '.join(n) for n in news_content]

In [29]:
news_content[:10]

['此外 自本周 6月12日 起 除小米手机6等15款机型外 其余机型已暂停更新发布 含开发版 体验版内测 稳定版暂不受影响 以确保工程师可以集中全部精力进行系统优化工作 有人猜测这也是将精力主要用到MIUI 9的研发之中 MIUI 8去年5月发布 距今已有一年有余 也是时候更新换代了 当然 关于MIUI 9的确切信息 我们还是等待官方消息',
 '骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器 高通强调 不会因为只考虑性能而去屏蔽掉小核心 相反 他们正联手微软 找到一种适合桌面平台的 兼顾性能和功耗的完美方案 报道称 微软已经拿到了一些新的源码 以便Windows 10更好地理解big little架构 资料显示 骁龙835作为一款集成了CPU GPU 基带 蓝牙 Wi Fi的SoC 比传统的Wintel方案可以节省至少30 的PCB空间 按计划 今年Q4 华硕 惠普 联想将首发骁龙835 Win10电脑 预计均是二合一形态的产品 当然 高通骁龙只是个开始 未来也许还能见到三星Exynos 联发科 华为麒麟 小米澎湃等进入Windows 10桌面平台',
 '此前的一加3T搭载的是3400mAh电池 DashCharge快充规格为5V 4A 至于电池缩水 可能与刘作虎所说 一加手机5要做市面最轻薄大屏旗舰的设定有关 按照目前掌握的资料 一加手机5拥有5 5寸1080P三星AMOLED显示屏 6G 8GB RAM 64GB 128GB ROM 双1600万摄像头 备货量 惊喜 根据京东泄露的信息 一加5起售价是xx99元 应该是在2799 2899 2999中的某个',
 '这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车 新华社记者张立云摄',
 '原标题 44岁女子跑深圳约会网友被拒 暴雨中裸身奔走 深圳交警微博称 昨日清晨交警发现有一女子赤裸上身 行走在南坪快速上 期间还起了轻生年头 一辅警发现后赶紧为其披上黄衣 并一路劝说她 那么事发时 到底都发生了些什么呢 南都记者带您一起还原现场 南都记者在龙岗大队坂田中队见到了辅警刘青 发现女生的辅警 一位外表高大帅气 说话略带些腼腆的90后青年 刘青介绍 6月16日早上7时36分 他正在环城南路附近值勤 接到中队关于一位女子裸身进入机动车可能有危险的警情 随后骑着小铁骑开始沿路

In [30]:
news_content = [cut(n) for n in news_content]

In [26]:
news_content[1]

'骁龙 835 作为 唯一 通过 Windows   10 桌面 平台 认证 的 ARM 处理器   高通 强调   不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心   相反   他们 正 联手 微软   找到 一种 适合 桌面 平台 的   兼顾 性能 和 功耗 的 完美 方案   报道 称   微软 已经 拿到 了 一些 新 的 源码   以便 Windows   10 更好 地 理解 big   little 架构   资料 显示   骁龙 835 作为 一款 集成 了 CPU   GPU   基带   蓝牙   Wi   Fi 的 SoC   比 传统 的 Wintel 方案 可以 节省 至少 30   的 PCB 空间   按计划   今年 Q4   华硕   惠普   联想 将 首发 骁龙 835   Win10 电脑   预计 均 是 二合一 形态 的 产品   当然   高通 骁龙 只是 个 开始   未来 也许 还 能 见到 三星 Exynos   联发科   华为 麒麟   小米 澎湃 等 进入 Windows   10 桌面 平台'

### Get similar words using word2vec model

In [ ]:
# can skip this and start from the next cell when re-running the code!

with open('sentences-cut.txt', 'w') as f:
    for n in news_content_words:
        f.write(n + '\n')
        

In [2]:
line_setences_path = '/Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/data/sentences-cut.txt'

In [3]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [4]:
news_word2vec= Word2Vec(LineSentence(line_setences_path), size=50, workers=8)

# Parameters:
# --- size: the dimensionality of the vector, or the size of the NN layers, 
#     which correspond to the “degrees” of freedom the training algorithm has.
#     Larger size would require more training data!
#     The default value for size is 100

#  See more clarification: 
#  https://stackoverflow.com/questions/45444964/python-what-is-the-size-parameter-in-gensim-word2vec-model-class
#  https://rare-technologies.com/word2vec-tutorial/ 

In [5]:
news_word2vec.most_similar('葡萄牙', topn=20)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('意大利', 0.8021515011787415),
 ('克罗地亚', 0.7992252707481384),
 ('捷克', 0.7990432381629944),
 ('奥地利', 0.7975884079933167),
 ('拉脱维亚', 0.788984477519989),
 ('伊斯坦布尔', 0.7777963280677795),
 ('比利时', 0.7773072123527527),
 ('摩洛哥', 0.7638806104660034),
 ('罗马尼亚', 0.7611976265907288),
 ('乌拉圭', 0.7520232796669006),
 ('西班牙', 0.7469848394393921),
 ('瑞典', 0.7392330765724182),
 ('比什凯克', 0.7391543984413147),
 ('丹麦', 0.7373787760734558),
 ('巴林', 0.7372497320175171),
 ('巴塞罗那', 0.7343917489051819),
 ('苏格兰', 0.7341640591621399),
 ('匈牙利', 0.7325829863548279),
 ('科特迪瓦', 0.7325041890144348),
 ('斯洛伐克', 0.7295225262641907)]

In [6]:
news_word2vec.most_similar('意大利', topn=20)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('捷克', 0.8346093893051147),
 ('葡萄牙', 0.8021515011787415),
 ('奥地利', 0.7748426795005798),
 ('德国', 0.75968337059021),
 ('比利时', 0.757101833820343),
 ('匈牙利', 0.7558014392852783),
 ('西班牙', 0.7485629916191101),
 ('瑞士', 0.7426245212554932),
 ('拉脱维亚', 0.7402303218841553),
 ('摩洛哥', 0.7388877868652344),
 ('罗马', 0.737281858921051),
 ('罗马尼亚', 0.7232054471969604),
 ('苏格兰', 0.7204700112342834),
 ('乌克兰', 0.7180824279785156),
 ('澳大利亚', 0.7102748155593872),
 ('基辅', 0.7101351618766785),
 ('保加利亚', 0.7064990997314453),
 ('丹麦', 0.7064815759658813),
 ('波兰', 0.6983714699745178),
 ('马德里', 0.6975508332252502)]

In [7]:
news_word2vec.most_similar('捷克', topn=20)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('罗马尼亚', 0.8712990283966064),
 ('奥地利', 0.835448145866394),
 ('意大利', 0.8346093893051147),
 ('拉脱维亚', 0.8137732148170471),
 ('匈牙利', 0.8105018138885498),
 ('葡萄牙', 0.7990432381629944),
 ('丹麦', 0.7982706427574158),
 ('克罗地亚', 0.7786042094230652),
 ('中国香港', 0.7640799880027771),
 ('基辅', 0.7628160119056702),
 ('乌克兰', 0.7619959115982056),
 ('斯洛文尼亚', 0.7562117576599121),
 ('保加利亚', 0.75162672996521),
 ('比利时', 0.7454712390899658),
 ('南非', 0.7453896403312683),
 ('波兰', 0.743989109992981),
 ('巴拉圭', 0.7420711517333984),
 ('摩洛哥', 0.7340573668479919),
 ('拉帕尔马', 0.7271416783332825),
 ('立陶宛', 0.717602550983429)]

In [8]:
news_word2vec.most_similar('罗马尼亚', topn=30)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('捷克', 0.8712990283966064),
 ('奥地利', 0.7987017035484314),
 ('丹麦', 0.7870330214500427),
 ('匈牙利', 0.7669995427131653),
 ('克罗地亚', 0.7658096551895142),
 ('中国香港', 0.7621965408325195),
 ('葡萄牙', 0.7611976265907288),
 ('斯洛文尼亚', 0.7533788084983826),
 ('南非', 0.7359355092048645),
 ('乌克兰', 0.7336846590042114),
 ('拉脱维亚', 0.7326607704162598),
 ('爱沙尼亚', 0.7290798425674438),
 ('吉尔吉斯斯坦', 0.7269991040229797),
 ('意大利', 0.7232054471969604),
 ('波兰', 0.7188212871551514),
 ('突尼斯', 0.7159653902053833),
 ('白俄罗斯', 0.7109165191650391),
 ('乌拉圭', 0.7101060748100281),
 ('澳大利亚', 0.7080951929092407),
 ('马耳他', 0.7079564332962036),
 ('比利时', 0.7056093811988831),
 ('索卡洛', 0.7047311067581177),
 ('古铁雷斯', 0.6987748742103577),
 ('津巴布韦', 0.6966949701309204),
 ('塞浦路斯', 0.6963719725608826),
 ('比什凯克', 0.6916817426681519),
 ('茨', 0.6910509467124939),
 ('拉帕尔马', 0.6902918219566345),
 ('保加利亚', 0.6875655055046082),
 ('纳米比亚', 0.6874528527259827)]

In [31]:
news_word2vec.most_similar('人工智能', topn=30)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('应用', 0.7608494758605957),
 ('量子', 0.7465195059776306),
 ('智能', 0.7440200448036194),
 ('互联网', 0.7396208047866821),
 ('虚拟现实', 0.738780677318573),
 ('百度', 0.7296438813209534),
 ('计算机', 0.7221457958221436),
 ('革命性', 0.7220436930656433),
 ('数字化', 0.7199282050132751),
 ('机器人', 0.7184781432151794),
 ('解决方案', 0.7161175012588501),
 ('技术', 0.7061582207679749),
 ('软件', 0.7037951350212097),
 ('信息技术', 0.6940164566040039),
 ('核心技术', 0.6902231574058533),
 ('发展趋势', 0.679265022277832),
 ('语音', 0.6790430545806885),
 ('机器', 0.6782774925231934),
 ('汽车行业', 0.6743263602256775),
 ('5G', 0.6734241247177124),
 ('商业化', 0.672935426235199),
 ('佳佳', 0.6669995784759521),
 ('业内', 0.6553199887275696),
 ('区块', 0.6536194682121277),
 ('北斗', 0.6534394025802612),
 ('算法', 0.6529671549797058),
 ('三星', 0.6480531692504883),
 ('智能手机', 0.6384239196777344),
 ('操作系统', 0.6359944939613342),
 ('颠覆性', 0.6354452967643738)]

In [32]:
news_word2vec.most_similar('佳佳', topn=30)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('凯利', 0.7538274526596069),
 ('凤凰网', 0.6839160323143005),
 ('人机对话', 0.6698284149169922),
 ('人工智能', 0.6669995784759521),
 ('答案', 0.6550537347793579),
 ('张艺谋', 0.6514453291893005),
 ('匪夷所思', 0.6445417404174805),
 ('肖骁', 0.6358044743537903),
 ('天方夜谭', 0.6165533661842346),
 ('黑嘉嘉', 0.6155110597610474),
 ('神药', 0.6153521537780762),
 ('王阳明', 0.6051992177963257),
 ('事物', 0.6046035885810852),
 ('剧情', 0.6038206219673157),
 ('观察家', 0.6011843681335449),
 ('有趣', 0.6010384559631348),
 ('张岩', 0.5994645357131958),
 ('综艺', 0.597311794757843),
 ('研究者', 0.5971728563308716),
 ('弗赖伊', 0.5969064235687256),
 ('李义东', 0.5955482721328735),
 ('霍金', 0.5926459431648254),
 ('影视剧', 0.5917186141014099),
 ('变脸', 0.5913613438606262),
 ('机器人', 0.5911327600479126),
 ('刘庆峰', 0.5911207795143127),
 ('人设', 0.5911023616790771),
 ('推理', 0.5906690359115601),
 ('小雅', 0.5885535478591919),
 ('波普尔', 0.5873091220855713)]

In [33]:
news_word2vec.most_similar('霍金', topn=30)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('外星人', 0.7587461471557617),
 ('研究者', 0.7560466527938843),
 ('凯利', 0.7379578948020935),
 ('谢泽雄', 0.731755256652832),
 ('沃森', 0.7260666489601135),
 ('心理学家', 0.7245619893074036),
 ('发表文章', 0.7232324481010437),
 ('假说', 0.7220876216888428),
 ('CNN', 0.7143890261650085),
 ('观察家', 0.7026435136795044),
 ('物理学家', 0.6991895437240601),
 ('谢泼德', 0.6913702487945557),
 ('弗赖伊', 0.6909985542297363),
 ('文正仁', 0.6872503757476807),
 ('刁', 0.6811679601669312),
 ('天文学家', 0.6793444156646729),
 ('罗夫', 0.6784137487411499),
 ('史丹', 0.6762403249740601),
 ('杨善林', 0.6752918362617493),
 ('哈钦斯', 0.6718539595603943),
 ('刘卫东', 0.6718029975891113),
 ('杨达卿', 0.6709124445915222),
 ('高福', 0.6691814064979553),
 ('凯文', 0.6642623543739319),
 ('古普塔', 0.663986325263977),
 ('爱台', 0.6609646677970886),
 ('麻省理工学院', 0.6604529619216919),
 ('古生物学家', 0.6589763164520264),
 ('倪光南', 0.658807098865509),
 ('牛津大学', 0.6562619209289551)]

In [34]:
news_word2vec.most_similar('外星人', topn=30)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('霍金', 0.7587461471557617),
 ('译者', 0.7583831548690796),
 ('幻想', 0.7313596606254578),
 ('谚语', 0.7295317649841309),
 ('弗赖伊', 0.722877025604248),
 ('爱台', 0.7189109325408936),
 ('笑话', 0.7131653428077698),
 ('亲中', 0.706908106803894),
 ('明日之星', 0.7062861323356628),
 ('台词', 0.7056949734687805),
 ('乔布斯', 0.7022846937179565),
 ('之死', 0.7004222273826599),
 ('史丹', 0.6995389461517334),
 ('那句', 0.698952317237854),
 ('调侃', 0.6972489953041077),
 ('野心', 0.69389408826828),
 ('旁人', 0.6931331753730774),
 ('子虚乌有', 0.6924238204956055),
 ('鸣叫声', 0.6911258101463318),
 ('侠盗', 0.6900421380996704),
 ('消息来源', 0.6886128187179565),
 ('翻译家', 0.6883841156959534),
 ('这篇', 0.6883050799369812),
 ('林全', 0.6877005696296692),
 ('没错', 0.6866949200630188),
 ('缩写', 0.6861328482627869),
 ('叫做', 0.6858958005905151),
 ('背影', 0.6831356883049011),
 ('望尘莫及', 0.6811276078224182),
 ('搞笑', 0.6808961629867554)]

In [9]:
news_word2vec.most_similar('说', topn=30)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('表示', 0.8959978222846985),
 ('指出', 0.8391335010528564),
 ('认为', 0.8304063081741333),
 ('坦言', 0.7886903285980225),
 ('告诉', 0.7840480804443359),
 ('看来', 0.7593174576759338),
 ('称', 0.7513463497161865),
 ('介绍', 0.747734785079956),
 ('明说', 0.7269032597541809),
 ('透露', 0.7131596207618713),
 ('所说', 0.6822411417961121),
 ('强调', 0.6732538342475891),
 ('文说', 0.6694759726524353),
 ('中说', 0.6428149342536926),
 ('时说', 0.6318460702896118),
 ('称赞', 0.6138980388641357),
 ('道', 0.5804609060287476),
 ('建议', 0.5734013319015503),
 ('提到', 0.5721763968467712),
 ('特别强调', 0.5712828636169434),
 ('说道', 0.5712825655937195),
 ('写道', 0.5556902289390564),
 ('地说', 0.5529962182044983),
 ('纳说', 0.5477250814437866),
 ('如是说', 0.5446681976318359),
 ('问', 0.5376904606819153),
 ('深有体会', 0.5322340130805969),
 ('的话', 0.5148422718048096),
 ('陈说', 0.5083231925964355),
 ('坦说', 0.508122444152832)]

In [10]:
news_word2vec.most_similar('认为', topn=30)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('指出', 0.904262900352478),
 ('表示', 0.8610953092575073),
 ('说', 0.8304063677787781),
 ('看来', 0.8137537240982056),
 ('称', 0.7826032042503357),
 ('坦言', 0.7739608883857727),
 ('普遍认为', 0.7603899836540222),
 ('建议', 0.7232547998428345),
 ('强调', 0.6947242617607117),
 ('透露', 0.680216908454895),
 ('告诉', 0.6688154339790344),
 ('所说', 0.6541367769241333),
 ('介绍', 0.5985261797904968),
 ('及其', 0.5957211852073669),
 ('相信', 0.5950856804847717),
 ('观点', 0.5849450826644897),
 ('而言', 0.5831488370895386),
 ('明说', 0.5807688236236572),
 ('对此', 0.5800443291664124),
 ('看法', 0.5774523019790649),
 ('猜测', 0.577131450176239),
 ('不明智', 0.5755384564399719),
 ('呼吁', 0.5741562843322754),
 ('担心', 0.5728529691696167),
 ('提到', 0.5714840888977051),
 ('事实上', 0.5704128742218018),
 ('声称', 0.5622197389602661),
 ('质疑', 0.5617133378982544),
 ('时说', 0.5605652332305908),
 ('资深', 0.5605463981628418)]

In [11]:
news_word2vec.most_similar('表明', topn=30)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('说明', 0.7921459674835205),
 ('事实上', 0.6684839129447937),
 ('反映', 0.6310827136039734),
 ('印证', 0.6234980225563049),
 ('结论', 0.6204043626785278),
 ('但', 0.6096541881561279),
 ('淡化', 0.6034464240074158),
 ('迹象', 0.6033262610435486),
 ('但是', 0.6018848419189453),
 ('显示', 0.5989968180656433),
 ('指向', 0.5957559943199158),
 ('意味着', 0.5946376919746399),
 ('提及', 0.5920984745025635),
 ('判断', 0.5915876030921936),
 ('释放', 0.5884886384010315),
 ('促使', 0.5871827602386475),
 ('推测', 0.5716366171836853),
 ('因此', 0.5707805156707764),
 ('是因为', 0.5703088045120239),
 ('现阶段', 0.5632461309432983),
 ('阐明', 0.5627313852310181),
 ('使得', 0.5622440576553345),
 ('尽管', 0.5557315945625305),
 ('孤立', 0.5517223477363586),
 ('普遍认为', 0.5515367984771729),
 ('现状', 0.5496326088905334),
 ('并非', 0.5487650632858276),
 ('阻碍', 0.5478509664535522),
 ('动向', 0.5469159483909607),
 ('削弱', 0.5449221134185791)]

Notes: 
* 数据量越多，同义词效果会越好（比如加入维基百科的数据）
* 寻找同义词的过程类似于一个搜索树：
      + 葡萄牙 --> 意大利， 摩洛哥，捷克， ...
      + 意大利 --> 葡萄牙，西班牙，比利时，捷克，...
      + 捷克 --> 罗马尼亚，拉脱维亚，丹麦, ...
      + 在这个搜索树里，单词出现频次越多，就和原有单词的意思越接近


按照之前我们熟悉的地图的搜索树的算法结构，可以得到初步的函数如下：

In [ ]:
def get_related_words(words, model):
    """
    @words are the initial words whose synonyms we want to search for
    @model is the word2vec model
    """
    unseen = [words]
    
    seen = set()
    
    while unseen:
        node = unseen.pop(0)
        new_branches = [w for w, s in model.most_similar(node, topn=30)]
        unseen += new_branches
        
        seen.add(node)
    
    return seen
 

但同义词搜索树与地图搜索的区别在于，我们想要知道每个单词出现的频次，以便对所有可能的单词进行排序，从而得到高频次的同义词；我们可以对原程序的 `seen` 变量进行修改：

In [35]:
from collections import defaultdict
def get_related_words(words, model):
    """
    @words are the initial words whose synonyms we want to search for
    @model is the word2vec model
    """
    unseen = words
    
    seen = defaultdict(int)
    
    max_size = 1000 # set a size limit for seen to reduce the searching space
    
    while unseen and len(seen) < max_size:
        if len(seen) % 50 == 0:
            print('seen length: {}'.format(len(seen)))
            
        node = unseen.pop(0)
        new_branches = [w for w, s in model.most_similar(node, topn=30)]
        unseen += new_branches
        
        seen[node] += 1
    
    return seen
    

程序尚待优化的点：
* `seen[node] +=1` 的计数方法可以进行优化，调整单词权重
* 加入dynamic programming的思路，将重复计算的分支结果存储起来，减少总体计算时间

In [36]:
len(news_word2vec.wv.vocab)

97927

In [37]:
related_words = get_related_words(['说', '建议'], news_word2vec)

# 注： 这其实是一个weak supervised-learning的思路

seen length: 0
seen length: 50


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


seen length: 100
seen length: 100
seen length: 100
seen length: 100
seen length: 100
seen length: 100
seen length: 100
seen length: 100
seen length: 100
seen length: 150
seen length: 200
seen length: 200
seen length: 200
seen length: 250
seen length: 300
seen length: 350
seen length: 400
seen length: 450
seen length: 450
seen length: 450
seen length: 500
seen length: 550
seen length: 550
seen length: 600
seen length: 650
seen length: 650
seen length: 700
seen length: 700
seen length: 750
seen length: 800
seen length: 850
seen length: 900
seen length: 950


In [38]:
# now we want to rank all the related words by frequency

sorted(related_words.items(), key=lambda x: x[1], reverse=True)

[('说', 24),
 ('指出', 24),
 ('所说', 24),
 ('表示', 23),
 ('坦言', 22),
 ('建议', 20),
 ('认为', 20),
 ('透露', 17),
 ('提到', 17),
 ('看来', 16),
 ('明说', 16),
 ('强调', 16),
 ('称', 15),
 ('告诉', 14),
 ('介绍', 12),
 ('中说', 12),
 ('说道', 12),
 ('时说', 11),
 ('特别强调', 11),
 ('呼吁', 11),
 ('对此', 11),
 ('普遍认为', 11),
 ('声称', 11),
 ('文说', 10),
 ('称赞', 10),
 ('如是说', 10),
 ('提出', 10),
 ('明确指出', 9),
 ('中称', 9),
 ('写道', 8),
 ('明确要求', 7),
 ('观点', 7),
 ('重申', 7),
 ('看法', 7),
 ('地说', 6),
 ('要求', 6),
 ('依据', 6),
 ('及其', 6),
 ('感慨', 6),
 ('评述', 6),
 ('明确提出', 6),
 ('问', 5),
 ('意见', 5),
 ('不明智', 5),
 ('资深', 5),
 ('直言', 5),
 ('说法', 5),
 ('解释', 5),
 ('深有体会', 4),
 ('坦说', 4),
 ('提醒', 4),
 ('采纳', 4),
 ('相应', 4),
 ('适当', 4),
 ('相信', 4),
 ('猜测', 4),
 ('质疑', 4),
 ('觉得', 4),
 ('问道', 4),
 ('还称', 4),
 ('他称', 4),
 ('证实', 4),
 ('提及', 4),
 ('澄清', 4),
 ('为什么', 4),
 ('主张', 4),
 ('阐述', 4),
 ('喜出望外', 4),
 ('国家有关', 4),
 ('明确规定', 4),
 ('决策', 4),
 ('明确', 4),
 ('规章', 4),
 ('现行', 4),
 ('起草', 4),
 ('立法', 4),
 ('纳说', 3),
 ('的话', 3),
 ('意见建议', 3),
 ('专家

In [39]:
related_words = get_related_words(['霍金', '外星人'], news_word2vec)

seen length: 0
seen length: 50


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


seen length: 100
seen length: 150
seen length: 150
seen length: 200
seen length: 200
seen length: 250
seen length: 250
seen length: 250
seen length: 300
seen length: 350
seen length: 400
seen length: 450
seen length: 500
seen length: 500
seen length: 550
seen length: 600
seen length: 650
seen length: 700
seen length: 750
seen length: 750
seen length: 800
seen length: 850
seen length: 850
seen length: 850
seen length: 900
seen length: 950


In [40]:
sorted(related_words.items(), key=lambda x: x[1], reverse=True)

[('霍金', 15),
 ('外星人', 12),
 ('麻省理工学院', 9),
 ('心理学家', 8),
 ('物理学家', 8),
 ('爱台', 7),
 ('古生物学家', 7),
 ('布林', 7),
 ('谢泽雄', 6),
 ('CNN', 6),
 ('弗赖伊', 6),
 ('凯文', 6),
 ('牛津大学', 6),
 ('亲中', 6),
 ('之死', 6),
 ('斯坦福大学', 6),
 ('研究者', 5),
 ('凯利', 5),
 ('罗夫', 5),
 ('史丹', 5),
 ('杨达卿', 5),
 ('笑话', 5),
 ('台词', 5),
 ('调侃', 5),
 ('旁人', 5),
 ('翻译家', 5),
 ('林全', 5),
 ('John', 5),
 ('卡斯', 5),
 ('霍马茨', 5),
 ('刘卫东', 4),
 ('古普塔', 4),
 ('倪光南', 4),
 ('那句', 4),
 ('鸣叫声', 4),
 ('背影', 4),
 ('类人猿', 4),
 ('克拉克', 4),
 ('主编', 4),
 ('比斯', 4),
 ('布拉德', 4),
 ('得克萨斯大学', 4),
 ('英国牛津大学', 4),
 ('潘建伟', 4),
 ('储殷', 4),
 ('贝纳姆', 4),
 ('神经科学', 4),
 ('东京大学', 4),
 ('沃尔特斯', 4),
 ('赖', 4),
 ('林斯', 4),
 ('闺蜜', 4),
 ('赖清德', 4),
 ('清德', 4),
 ('立委', 4),
 ('这番话', 4),
 ('马英九', 4),
 ('实话', 4),
 ('记叙', 4),
 ('沃森', 3),
 ('假说', 3),
 ('观察家', 3),
 ('天文学家', 3),
 ('杨善林', 3),
 ('哈钦斯', 3),
 ('高福', 3),
 ('译者', 3),
 ('幻想', 3),
 ('明日之星', 3),
 ('消息来源', 3),
 ('没错', 3),
 ('搞笑', 3),
 ('物理学', 3),
 ('计算机科学', 3),
 ('历史学', 3),
 ('凯伊', 3),
 ('卡伦', 3),
 ('陆奇', 3

In [41]:
related_words = get_related_words(['人工智能'], news_word2vec)

seen length: 0
seen length: 50
seen length: 50


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


seen length: 100
seen length: 100
seen length: 150
seen length: 200
seen length: 250
seen length: 250
seen length: 250
seen length: 300
seen length: 300
seen length: 350
seen length: 400
seen length: 400
seen length: 400
seen length: 400
seen length: 400
seen length: 400
seen length: 400
seen length: 400
seen length: 400
seen length: 450
seen length: 500
seen length: 500
seen length: 550
seen length: 550
seen length: 600
seen length: 650
seen length: 700
seen length: 700
seen length: 700
seen length: 700
seen length: 700
seen length: 700
seen length: 700
seen length: 700
seen length: 750
seen length: 750
seen length: 800
seen length: 800
seen length: 850
seen length: 850
seen length: 900
seen length: 950
seen length: 950
seen length: 950
seen length: 950
seen length: 950


In [42]:
sorted(related_words.items(), key=lambda x: x[1], reverse=True)

[('智能', 51),
 ('操作系统', 51),
 ('虚拟现实', 50),
 ('应用', 49),
 ('人工智能', 47),
 ('算法', 46),
 ('量子', 42),
 ('语音', 42),
 ('互联网', 41),
 ('集成', 41),
 ('高性能', 41),
 ('信息技术', 40),
 ('智能化', 39),
 ('北斗', 36),
 ('模块', 34),
 ('无线', 34),
 ('软件', 32),
 ('传感器', 32),
 ('网联', 32),
 ('智能手机', 31),
 ('数字化', 30),
 ('高精度', 30),
 ('芯片', 29),
 ('计算机', 28),
 ('技术', 27),
 ('处理器', 26),
 ('解决方案', 25),
 ('移动', 25),
 ('互联', 25),
 ('三维', 25),
 ('柔性', 25),
 ('供应链', 24),
 ('成像', 24),
 ('核心技术', 23),
 ('光子', 22),
 ('联网', 22),
 ('服务商', 22),
 ('内置', 22),
 ('新一代', 22),
 ('百度', 21),
 ('商业化', 21),
 ('粒子', 21),
 ('人脸', 21),
 ('3D', 21),
 ('车载', 21),
 ('网络化', 20),
 ('虚拟', 20),
 ('复合材料', 20),
 ('导航', 19),
 ('技术创新', 19),
 ('GPS', 19),
 ('云', 18),
 ('可视化', 18),
 ('ERP', 18),
 ('开放平台', 18),
 ('IoT', 18),
 ('智能家居', 17),
 ('OLED', 17),
 ('电子', 17),
 ('人脸识别', 17),
 ('通信', 17),
 ('关键技术', 16),
 ('运算', 16),
 ('服务器', 16),
 ('机器人', 15),
 ('机器', 15),
 ('汽车行业', 15),
 ('AR', 15),
 ('运营商', 15),
 ('高通', 15),
 ('技术手段', 15),
 ('物', 15),
 ('宽带', 15),
 

### Keyword Extraction
+ calculating TF-IDF;
+ TF-IDF vectorized;
+ visualization - word cloud

Suppose we have a collection of $N$ documents, and for a given document $d$, the frequency of term $t$ appeared in a specific document, $d$, is $tf_{t,d}$ (*term frequency*), and for all documents in the collection, the number of documents containing the specified term $t$ is $df_t$ (*document frequency*), then we define the following:

the ***inverse document frequency*** is given by:
$$idf_t = log \frac{N}{df_t}$$

and therefore ***TF-iDF*** is given by:
$$tf-idf_{t,d} = df_{t,d} \times idf_t$$

In [43]:
def document_frequency(word, list_text): 
    "Returns the document frequency of a @word in a given collection of texts, @list_text"
    return sum(1 for text in list_text if word in text)

In [44]:
document_frequency('的', news_content)

70342

In [46]:
document_frequency('火星', news_content)

116

In [47]:
document_frequency('人工智能', news_content)

558

In [48]:
document_frequency('霍金', news_content)

53

In [49]:
document_frequency('外星人', news_content)

19

In [50]:
import math
def idf(word, list_text):
    """Get the inversed document frequency of a @word in a collection of texts, @list_text"""
    return math.log10(len(list_text) / document_frequency(word, list_text))

In [51]:
idf('的', news_content)

0.1051466115514474

In [52]:
idf('火星', news_content)

2.887903334565555

In [54]:
idf('小米', news_content)

# the commonplace word,'的', should have a lower idf than more important words such as '小米' 

2.948039950009831

In [55]:
idf('外星人', news_content)

3.6736077228396447

In [56]:
def tf(word, document):
    """
    Get the term frequemcy of a @word in a @document.
    """
    words = document.split()
    
    return sum(1 for w in words if w == word)

In [57]:
tf('银行', news_content[11])

6

In [58]:
tf('创业板', news_content[11])

6

In [60]:
tf('短期', news_content[11])

3

In [62]:
tf('市场', news_content[11])

20

In [63]:
# now we can define a function that returns 
# the rank of tf-idf values for all words appeared in the document,
# given a collection of documents

def get_word_rank_in_a_document(index, document_collection):
    """
    @index = an integer specify the index of the document in the callableollection;
    @document_collection = a list of documents to be searched
    """
    document = document_collection[index]
    words = set(document.split())
    
    tfidf = [
        (w, tf(w, document) * idf(w, document_collection)) for w in words
    ]
    
    tfidf = sorted(tfidf, key=lambda x: x[1], reverse=True)
    
    return tfidf

In [64]:
get_word_rank_in_a_document(11, news_content)

[('市场', 21.353584391728972),
 ('股指', 18.198034968575843),
 ('周四', 17.26088617439966),
 ('均线', 15.505514875366993),
 ('板块', 15.184208429020511),
 ('创业板', 15.040542723113257),
 ('沪', 14.096891190311872),
 ('反弹', 11.40131732928378),
 ('巨丰', 11.244724023409647),
 ('普涨', 11.1657372072426),
 ('居前', 10.78835527475667),
 ('午后', 10.712813898115176),
 ('早盘', 10.614032989531069),
 ('大盘', 10.528860150725679),
 ('保险', 9.712428450401568),
 ('跳水', 9.392541082832015),
 ('具备', 9.384071811999714),
 ('局部性', 8.950480138145622),
 ('走势', 8.886316192504337),
 ('回落', 8.85749598983617),
 ('银行', 8.730101656649362),
 ('大涨', 8.113164172292002),
 ('涨幅', 7.982850978349081),
 ('阴线', 7.904722647584947),
 ('普跌', 7.676835942971274),
 ('半年线', 7.552540129473584),
 ('上影线', 7.552540129473584),
 ('题材', 7.529644664184074),
 ('个股', 7.317431171173604),
 ('伏击', 7.0747759516433115),
 ('探底', 7.010406584900509),
 ('行情', 6.978062869252294),
 ('兴业银行', 6.560526931713512),
 ('沪市', 6.52433048752792),
 ('复星', 6.472715960315349),
 ('白马股'

In [68]:
get_word_rank_in_a_document(101, news_content)

[('机械', 19.73268423154147),
 ('工业', 13.726799287798574),
 ('百强', 12.218936930008129),
 ('机械行业', 11.732905915902746),
 ('中机联', 9.904722647584947),
 ('王瑞祥', 9.904722647584947),
 ('增速', 8.057368905265053),
 ('态势', 7.971371787277902),
 ('恢复性', 6.867694767829172),
 ('规模', 6.861746056315247),
 ('回落', 6.643121992377128),
 ('增长', 6.604587773896507),
 ('企业', 6.260042817487905),
 ('陈斌', 5.9311791790524575),
 ('呈现', 5.279381611367784),
 ('原材料', 5.215938100214726),
 ('16992', 4.9523613237924735),
 ('但远', 4.9523613237924735),
 ('刘瑾', 4.651331328128492),
 ('三十', 4.6438668975349),
 ('入围', 4.629743188559925),
 ('钢铁', 4.588699854270723),
 ('回升', 4.553165964236777),
 ('联合会', 4.4415451172114695),
 ('行业', 4.256149034981132),
 ('后移', 4.253391319456455),
 ('前十强', 4.253391319456455),
 ('发展', 4.24347205022841),
 ('亿元', 4.194707226838383),
 ('更新改造', 4.17421007340883),
 ('机械类', 4.17421007340883),
 ('相当规模', 4.17421007340883),
 ('年降', 4.107263283778217),
 ('徐工', 3.9981188143531488),
 ('工程', 3.9916862993694178),
 

### TFIDF Vectorizezd

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

### Word cloud

In [69]:
import wordcloud

### Build a Search Engine